<a href="https://colab.research.google.com/github/Davjot/PlantDoc/blob/main/Image_plantdoc_IRV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import os
from tensorflow.keras import layers
from tensorflow.keras import Model

In [31]:
!wget --no-check-certificate \
   https://storage.googleapis.com/tensorflow/keras-applications/inception_resnet_v2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5

--2023-04-05 15:49:44--  https://storage.googleapis.com/tensorflow/keras-applications/inception_resnet_v2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.128, 142.250.141.128, 2607:f8b0:4023:c0d::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 219055592 (209M) [application/octet-stream]
Saving to: ‘inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5’

inception_resnet_v2 100%[===================>] 208.91M   248MB/s    in 0.8s    

2023-04-05 15:49:45 (248 MB/s) - ‘inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [219055592/219055592]



In [32]:
from tensorflow.keras.applications import InceptionResNetV2

local_weights_file = '/content/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionResNetV2(
    input_shape=(299, 299, 3), include_top=False, weights='imagenet')
pre_trained_model.load_weights(local_weights_file)

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
base_dir = '/content/drive/MyDrive/Plantdoc'
train_dir = os.path.join(base_dir)
validation_dir = os.path.join(base_dir)

In [35]:
train_apple_rust_leaf_dir = os.path.join(train_dir, 'Apple rust leaf')
validation_apple_rust_leaf_dir = os.path.join(validation_dir, 'Apple rust leaf')
train_blueberry_leaf_dir = os.path.join(train_dir, 'Blueberry leaf')
validation_blueberry_leaf_dir = os.path.join(validation_dir, 'Blueberry leaf')
#train_apple_rust_leaf_fnames = os.listdir(train_apple_rust_leaf_dir) 
#training set
train_apple_leaf_dir = os.path.join(train_dir, 'Apple leaf')
train_apple_scab_leaf_dir = os.path.join(train_dir, 'Apple scab leaf')
train_bell_pepper_leaf_dir = os.path.join(train_dir, 'Bell pepper leaf')
train_bell_pepper_leaf_spot_dir = os.path.join(train_dir, 'Bell pepper leaf spot')
train_cherry_leaf_dir = os.path.join(train_dir, 'Cherry leaf')
train_corn_gray_leaf_spot_dir = os.path.join(train_dir, 'Corn Gray leaf spot')

#validation set
validation_apple_leaf_dir = os.path.join(validation_dir, 'Apple leaf')
validation_apple_scab_leaf_dir = os.path.join(validation_dir, 'Apple scab leaf')
validation_bell_pepper_leaf_dir = os.path.join(validation_dir, 'Bell pepper leaf')
validation_bell_pepper_leaf_spot_dir = os.path.join(validation_dir, 'Bell pepper leaf spot')
validation_cherry_leaf_dir = os.path.join(validation_dir, 'Cherry leaf')
validation_corn_gray_leaf_spot_dir = os.path.join(validation_dir, 'Corn Gray leaf spot')

In [36]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Note that the validation data should not be augmented!
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir, # This is the source directory for training images
        target_size=(299, 299),  # All images will be resized to 150x150
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

# Flow validation images in batches of 20 using val_datagen generator
validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(299, 299),
        batch_size=32,
        class_mode='categorical')

Found 80 images belonging to 8 classes.
Found 80 images belonging to 8 classes.


In [37]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [38]:
# create the base pre-trained model
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape = (299, 299, 3))

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = layers.Dropout(0.2)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(8, activation='softmax')(x)

In [39]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [40]:
from tensorflow.keras.optimizers import Adam
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['acc'])

In [41]:
import tensorflow as tf
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2),
    tf.keras.callbacks.ModelCheckpoint(filepath='/content/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5', save_weights_only = True),
    tf.keras.callbacks.TensorBoard(log_dir='/content/drive/MyDrive/logdir_plantdoc_irv2'),
]

In [42]:
history = model.fit(
      train_generator,
      epochs=5,
      validation_data=validation_generator,
      verbose=2,
      callbacks = [my_callbacks])

Epoch 1/5
3/3 - 155s - loss: 2.3769 - acc: 0.1375 - val_loss: 1.8135 - val_acc: 0.5000 - 155s/epoch - 52s/step
Epoch 2/5
3/3 - 129s - loss: 2.0555 - acc: 0.3250 - val_loss: 1.5894 - val_acc: 0.4875 - 129s/epoch - 43s/step
Epoch 3/5
3/3 - 131s - loss: 1.6654 - acc: 0.4125 - val_loss: 1.4126 - val_acc: 0.5875 - 131s/epoch - 44s/step
Epoch 4/5
3/3 - 129s - loss: 1.4485 - acc: 0.5000 - val_loss: 1.2657 - val_acc: 0.5500 - 129s/epoch - 43s/step
Epoch 5/5
3/3 - 91s - loss: 1.1706 - acc: 0.6000 - val_loss: 1.0870 - val_acc: 0.6125 - 91s/epoch - 30s/step
